In [3]:
from __future__ import division, print_function, unicode_literals
from io import open

import numpy as np 
import os 

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
    
%matplotlib inline 
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize']= 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

In [4]:
def plot_image(image):
    plt.imshow(image, cmap="grey", interpolation="nearest")
    plt.axis("off")
    
def plor_color_image(image):
    plt.imshow(image.astype(np.uint8),interpolation="nearest")
    plt.axis("off")

In [5]:
import tensorflow as tf